## Cleaning of Datasets

####  1) Pull in Data from exploratory file

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
scheme_users = pd.read_csv("codeclan/scheme_users.csv")

scheme_jigs = pd.read_csv("codeclan/schemejigs.csv")

simd = pd.read_csv("codeclan/simd.csv")

jigger_details = pd.read_csv("codeclan/jiggerdetails.csv")

jigger_follow_ups = pd.read_csv("codeclan/jigger_follow_ups.csv")


#### 2) Dropping variables that not adding value

In [ ]:
jigger_details_wip = jigger_details.drop(['email', 'firstName', 'lastName', 'address', 'city', 'postcode', 'simd_id', 'orig_scheme_id', 'optinfo', 'ageGroup_id', 'dataprotection', 'ethnicOrigin', 'landline', 'mobile', 'followupPermission',
                                         'latestfollowup', 'followup_sent', 'followup_sent_date', 'followup_done', 'followup_remind_sent',
                                        'last_jig_id', 'train_date_train', 'is_trained', 'train_date_dementia', 'train_date_refresh','train_date_firstaid','cohort_local', 'eNews', 'email_footnotes', 'post_footnotes',
       'newfixesdone', 'safecode', 'regcode', ], axis=1)

In [ ]:
#take only 2012 onwards
import datetime
jigger_details_wip['year'] = pd.to_datetime(jigger_details_wip.joinDate, errors ="coerce" )
jigger_details_wip['year'] = jigger_details_wip['year'].dt.year.astype('Int64')



#### 3) Pull in local authority anf 'friendliness' data to jigger_details

In [ ]:
scheme_users.drop(scheme_users.columns.difference(['schemeID', 'friendly', 'localAuthority']), 1, inplace = True)

In [ ]:
# adjust column name for simpler join
scheme_users.rename(columns = {"schemeID" : "scheme_id"}, inplace = True)

In [ ]:
jigger_details_wip = pd.merge(jigger_details_wip, scheme_users, how='left', on=['scheme_id'])

In [ ]:
jigger_details_wip[jigger_details_wip.localAuthority.isna()]

In [ ]:
jigger_details_wip = jigger_details_wip.drop(['scheme_id'], axis=1)

##### 4) Pull in relevant SIMD data - and text removal from string
The SIMD Id  columns in databases given are not correctly recorded so in the end I coul not use the below SIMD data

In [ ]:
# actually, this wasnt possible as the SIMD id codes given in the data are incorrect.  Max should be 6976 - 25000 go above 6976
# Only really wanted URname Can still take value from Working Age Population / Population - but not critical as lots more data to work back - will feedback to charity
#simd.columns

In [ ]:
#simd['DZ'] = simd['DZ'].map(lambda x: x.lstrip('S0100'))

In [ ]:
#simd_wip = simd.drop(simd.columns.difference(['DZ', 'Population', 'Working_Age_Population', 'URname']), 1)

In [ ]:
#simd_wip.URname.unique()

In [ ]:
# adjust column name for simpler join
#simd_wip.rename(columns = {"DZ" : "simd_id"}, inplace = True)

In [ ]:
#jigger_details_wip['simd_id'] = jigger_details_wip['simd_id'].astype(str)

In [ ]:
#jigger_details_wip = pd.merge(jigger_details_wip, simd_wip , how='left', on=['simd_id']) 

### Still Jigging - variable for use later

In [ ]:
#Keep for use later
still_Jigging = jigger_follow_ups.drop(jigger_follow_ups.columns.difference(['id', 'stilljigging']), 1)
still_Jigging.rename(columns = {"id" : "userID"}, inplace = True)
still_Jigging.to_csv ('codeclan/still_Jigging.csv', index = False, header=True)

##### 5) Combine Anxiety and Depression into one column

In [ ]:
jigger_details_wip.columns

In [ ]:
jigger_details_wip['mental_health'] = (jigger_details_wip[['mentalHealth', 'anxietyDepression']] == 1).any(axis = 1).astype(int)

In [ ]:
#checking numbers listed as having mental health problems in database
jigger_details_wip.mental_health.sum()

In [ ]:
#dropthese columns now have new column
jigger_details_wip = jigger_details_wip.drop(['anxietyDepression', 'mentalHealth'], axis=1)

#### 6) from here on I'm going to treat -1 in health issues (NAN) as 0 ... reason being I'm taking it user would have entered if relevant.  Focus is on the 1s

In [ ]:
jigger_details_wip[['heartCondition', 'chestPainActive', 'chestPainOther',
       'looseBalanceConsciousness', 'boneJointProblem', 'diabetes', 'asthma',
       'bloodPressure', 'disabled', 'overweight', 'cancer', 'dementia', 'copd']] = jigger_details_wip[['heartCondition', 'chestPainActive', 'chestPainOther',
       'looseBalanceConsciousness', 'boneJointProblem', 'diabetes', 'asthma',
       'bloodPressure', 'disabled', 'overweight', 'cancer', 'dementia', 'copd']].replace([-1],0)

In [ ]:
#Checking for asthm before I amalgamate 
jigger_details_wip.asthma.sum()

##### 7) I'll now take what value I can from other health problems - as some are listed often or could be almagamated to existing variables

In [ ]:
jigger_details_wip.otherProblems= jigger_details_wip['otherProblems'].astype(str)

In [ ]:
# Will look through the below and look for 
from collections import Counter
results = Counter()
jigger_details_wip.otherProblems.str.lower().str.split().apply(results.update)
print(results)

In [ ]:
## function created for colums to add
def str_check(mylist):
    #add or operator between list items
    pattern = '|'.join(mylist)
    x = jigger_details_wip['otherProblems'].str.contains(pattern).astype(int) 
    return x

jigger_details_wip['arthritis'] = str_check(['arthritis', 'arithtitis',  'osteo', 'rheu'])
jigger_details_wip['visually_impaired'] = str_check(['blind', 'visu',  'sight'])
jigger_details_wip['thyroid_issues'] = str_check(['thyr'])
jigger_details_wip['hearing_issues'] = str_check(['deaf', 'hear'])

jigger_details_wip['asthma_2'] = str_check(['asthma'])
jigger_details_wip['disabled_2'] = str_check(['disability'])
jigger_details_wip['copd_2'] = str_check(['copd'])
jigger_details_wip['mental_health_2'] = str_check(['depress', 'stress', 'anxiety'])
jigger_details_wip['balance_2'] = str_check(['balance', 'dizzi', 'vertigo'])
jigger_details_wip['dementia_2'] = str_check(['dementia'])



In [ ]:
# functoipn to pull two columns in to one
def update_health(current, update):
    return (jigger_details_wip[[current, update]] == 1).any(axis = 1).astype(int)
    
    
jigger_details_wip['asthma'] = update_health('asthma', 'asthma_2')
jigger_details_wip['disabled'] = update_health('disabled', 'disabled_2')
jigger_details_wip['copd'] = update_health('copd', 'copd_2')
jigger_details_wip['mental_health'] = update_health('mental_health', 'mental_health_2')
jigger_details_wip['looseBalanceConsciousness'] = update_health('looseBalanceConsciousness', 'balance_2')
jigger_details_wip['dementia'] = update_health('dementia', 'dementia_2')



In [ ]:
# Check logic worked
# jigger_details_wip[['asthma', 'asthma_2']].tail(50)

In [ ]:
#drop the update cells
jigger_details_wip = jigger_details_wip.drop(['asthma_2', 'disabled_2', 'copd_2', 'mental_health_2', 'balance_2', 'dementia_2'], axis=1)

In [ ]:
# checking asthma total as it should be
jigger_details_wip.asthma.sum()

##### 8) Add other Health Problems as binary
now that I've extracted and amalgamated data from it

In [ ]:
# convert nans to 0
jigger_details_wip['other_health_problems'] = np.where(jigger_details_wip['otherProblems'].str.contains("nan"), 0, 1)

In [ ]:
# drop otherProblems as data value has been extracted
jigger_details_wip = jigger_details_wip.drop(['otherProblems'], axis=1)

#### 9) Pivot wide to long - jigger health issues combined / pivotted

In [ ]:
jigger_details_piv1 = pd.melt(jigger_details_wip,id_vars=['id', 'year', 'scheme', 'localAuthority', 'is_current_user', 'simd_d','toLooseWeight', 'toMeetNewPeople', 'toFeelHealthier',
       'toVolunteer', 'preferJigging', 'otherJoinReason', 'gpReferral',
       'healthProfReferral', 'leafletReferral', 'FBReferral', 'posterReferral',
       'familyFriendsReferral', 'newsPaperReferral', 'otherReferral', 'ageGroup', 'employmentStatus', 'sex', 'joinDate',
       'last_jig_date', 'deceased', 'jigs_count', 'past_week_30_initial', 'past_week_30_final', 'currentlyRegularActive',
       'currentlyRegularActiveFinal', 'followup_received_date', 'volunteer', 'pedometer', 'diff_jigs_count', 'friendly'],var_name='health_issues',value_name='health_value')

In [ ]:
# have a look at one unique id as a check
# jigger_details_piv1[jigger_details_piv1['id'] == 4]

##### 10) Add Has Health Issues summary column

In [ ]:
# selected columns, filter and rename before joining back in
health_col = jigger_details_piv1.drop(jigger_details_piv1.columns.difference(['health_value', 'id']), 1)
health_col = health_col[health_col['health_value'] == 1]
health_col.rename(columns={'health_value':'has_health_problem'}, inplace = True)

In [ ]:
jigger_details_wip = pd.merge(jigger_details_wip, health_col , how='left', on= 'id')
jigger_details_wip['has_health_problem'] = jigger_details_wip['has_health_problem'].astype(str)
jigger_details_wip['has_health_problem'] = np.where(jigger_details_wip['has_health_problem'].str.contains('0'), 'No', 'Yes')

In [ ]:
jigger_details_wip.drop_duplicates(inplace = True)

In [ ]:
jigger_details_piv1 = pd.merge(jigger_details_piv1, health_col , how='left', on= 'id')
jigger_details_piv1['has_health_problem'] = jigger_details_piv1['has_health_problem'].astype(str)
jigger_details_piv1['has_health_problem'] = np.where(jigger_details_piv1['has_health_problem'].str.contains('0'), 'No', 'Yes')
jigger_details_piv1.drop_duplicates(inplace = True)

##### 11) Pivot wide to long - Reason for signing up combined

In [ ]:
# Step 1 - pull out other referrals for possible reporting
jigger_details_wip.otherReferral= jigger_details_wip['otherReferral'].astype(str)
results = Counter()
jigger_details_wip.otherReferral.str.lower().str.split().apply(results.update)
print(results)

In [ ]:
jigger_details_wip.columns

#### Referrals extracted for use later

In [ ]:
referrals = jigger_details_wip.drop(['heartCondition',
       'chestPainActive', 'chestPainOther', 'looseBalanceConsciousness',
       'boneJointProblem', 'diabetes', 'asthma', 'bloodPressure','disabled', 'overweight', 'ageGroup', 'employmentStatus', 'sex', 'followup_received_date', 'copd', 'volunteer', 'cancer', 'dementia',
       'pedometer', 'deceased', 'jigs_count', 'last_jig_date',
       'diff_jigs_count', 'localAuthority', 'friendly', 'mental_health',
       'arthritis', 'visually_impaired', 'thyroid_issues', 'hearing_issues',
       'other_health_problems', 'toLooseWeight', 'toMeetNewPeople', 'toFeelHealthier', 'toVolunteer',
       'preferJigging', 'otherJoinReason'], axis = 1)

In [ ]:
referrals['otherReferral'] = np.where(referrals['otherReferral'].str.contains("nan"), 0, 1)

In [ ]:
referrals['otherReferral'].unique()

In [ ]:
referrals = pd.melt(referrals,id_vars=['id', 'scheme', 'is_current_user', 'simd_d',
       'joinDate', 'past_week_30_initial', 'past_week_30_final',
       'currentlyRegularActive', 'currentlyRegularActiveFinal', 'year',
       'has_health_problem'],var_name='referral_route',value_name='referral_value')

In [ ]:
referrals['referral_value'] = referrals['referral_value'].replace([-1],0)
referrals.to_csv('codeclan/referrals.csv', index = False, header=True)

In [ ]:
referrals.referral_value.unique()

#### Joining Reasons extracted for use later

In [ ]:
reasons = jigger_details_wip.drop(['heartCondition',
       'chestPainActive', 'chestPainOther', 'looseBalanceConsciousness',
       'boneJointProblem', 'diabetes', 'asthma', 'bloodPressure','disabled', 'overweight', 'ageGroup', 'employmentStatus', 'sex', 'followup_received_date', 'copd', 'volunteer', 'cancer', 'dementia',
       'pedometer', 'deceased', 'jigs_count', 'last_jig_date',
       'diff_jigs_count', 'localAuthority', 'friendly', 'mental_health',
       'arthritis', 'visually_impaired', 'thyroid_issues', 'hearing_issues',
       'other_health_problems', 'gpReferral', 'healthProfReferral',
       'leafletReferral', 'FBReferral', 'posterReferral',
       'familyFriendsReferral', 'newsPaperReferral', 'otherReferral'], axis=1)


In [ ]:
reasons.columns

In [ ]:
#there's too many joining reasons to pick at and little time so if other reason = 1, else 0
reasons['otherJoinReason'] = np.where(reasons['otherJoinReason'].str.contains("nan"), 0, 1)


In [ ]:
reasons['otherJoinReason'].unique()

In [ ]:
reasons = pd.melt(reasons,id_vars=['id', 'scheme', 'is_current_user', 'simd_d',
       'joinDate', 'past_week_30_initial', 'past_week_30_final',
       'currentlyRegularActive', 'currentlyRegularActiveFinal', 'year',
       'has_health_problem'],var_name='joining_reasons',value_name='reason_value')


In [ ]:
#replace -1s with 0s
reasons['reason_value'] = reasons['reason_value'].replace([-1],0)
reasons.to_csv('codeclan/joining_reasons.csv', index = False, header=True)

##### 12) Extract feedback

In [ ]:
# anychanges and follow_notes both own dataframes - reduced to two cols with same headings for appending
any_changes = jigger_follow_ups.drop(jigger_follow_ups.columns.difference(['anychanges']), 1)
any_changes.rename(columns = {"anychanges" : "feedback"}, inplace = True)
f_notes = jigger_follow_ups.drop(jigger_follow_ups.columns.difference(['follow_notes']), 1)
f_notes.rename(columns = {"follow_notes" : "feedback"}, inplace = True)

In [ ]:
# combine feedback data 
feedback = any_changes.append(f_notes).dropna()
feedback.head()

In [ ]:
# can now start process of extracting most common words
feedback_words = Counter()
feedback.feedback.str.lower().str.split().apply(feedback_words.update)
print(feedback_words)

In [ ]:
# using nltk and spacy packages to remove stopwords
import nltk
nltk.download('punkt') 

In [ ]:
from nltk.tokenize import word_tokenize as word_tokenize

In [ ]:
import spacy

In [ ]:

sp = spacy.load('en_core_web_sm')

all_stopwords = sp.Defaults.stop_words

In [ ]:
pip install wordcloud

In [ ]:
from wordcloud import WordCloud
from matplotlib import pyplot as plt

In [ ]:
comment_words = '' 
stopwords = sp.Defaults.stop_words 
  
# iterate through the csv file 
for val in feedback['feedback']: 
      
    # typecaste each val to string 
    val = str(val) 
  
    # split the value 
    tokens = val.split() 
      
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
      
    comment_words += " ".join(tokens)+" "
  
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(comment_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.savefig('images/feedback.png', dpi=300, bbox_inches='tight')
plt.show() 




In [ ]:
# extracted the most commonly used words from the feedback - psychological benefits as well as physical

##### 13) Join and Pivot Data for follow ups
remove columns that I wont be analysing against follow up

In [ ]:
jigger_follow_ups.columns

In [ ]:
# remove these now feedback extracted
jigger_follow_ups_wip = jigger_follow_ups.drop(['id','anychanges', 'follow_notes', 'scheme_id'], axis=1)

In [ ]:
# other activities engaged in
jigger_follow_ups_wip.other= jigger_follow_ups_wip['other'].astype(str)
results = Counter()
jigger_follow_ups_wip.other.str.lower().str.split().apply(results.update)
print(results)

In [ ]:
jigger_details_wip.rename(columns = {"id" : "userID"}, inplace = True)

In [ ]:
jigger_details_clean = jigger_details_wip.drop(jigger_details_wip.columns.difference(['userID', 'scheme', 'is_current_jigger', 'simd_d',  'ageGroup', 'employmentStatus', 'sex',
       'joinDate', 'past_week_30_initial', 'past_week_30_final',
       'currentlyRegularActive', 'currentlyRegularActiveFinal',
       'followup_received_date', 'localAuthority', 'year', 'has_health_problem']), 1,)


In [ ]:
jigger_follow_ups_wip.shape

In [ ]:
jigger_details_b4aft = pd.merge(jigger_follow_ups_wip , jigger_details_wip, how='left', on= 'userID')

-------------_______________-______-----------------

In [ ]:
jigger_details_b4aft.shape

In [ ]:
jigger_details_b4aft.columns

In [ ]:
jigger_details_wip.shape

In [ ]:
#pivot for new activities taken up
jigger_details_b4aft_piv1 = pd.melt(jigger_details_b4aft,id_vars=['userID',  'year', 'scheme', 'localAuthority', 'has_health_problem',
       'is_current_user', 'simd_d',
       'ageGroup', 'employmentStatus', 'sex', 'joinDate', 'last_jig_date',
       'past_week_30_initial', 'past_week_30_final',
        'currentlyRegularActive', 'currentlyRegularActiveFinal', 'volunteer', 'pedometer', 'jigs_count',
       'diff_jigs_count', 'deceased',
       'friendly', 'date',
       'nowshortjourneys', 'bustrain'],var_name='new_activity',value_name='activity_value')

In [ ]:
jigger_details_b4aft_piv1.shape

In [ ]:
# pivot - are they more likely to take public transport
jigger_details_b4aft_piv2 = pd.melt(jigger_details_b4aft_piv1,id_vars=['userID',  'year', 'scheme', 'localAuthority', 'has_health_problem',
       'is_current_user', 'simd_d',
       'ageGroup', 'employmentStatus', 'sex', 'joinDate', 'last_jig_date',
       'past_week_30_initial', 'past_week_30_final',
       'currentlyRegularActive', 'currentlyRegularActiveFinal', 'volunteer', 'pedometer', 'jigs_count',
       'diff_jigs_count', 'deceased',
       'friendly', 'date', 'new_activity', 'activity_value'],var_name='travel',value_name='travel_value')

In [ ]:
jigger_details_b4aft_piv1.head()

In [ ]:
jigger_details_b4aft.drop(jigger_details_b4aft.columns.difference(['userID',  'year', 'scheme', 'is_current_user', 'simd_d', 'ageGroup', 'employmentStatus', 'sex',
       'joinDate', 'past_week_30_initial', 'past_week_30_final',
       'currentlyRegularActive', 'currentlyRegularActiveFinal',
       'followup_received_date', 'localAuthority', 'has_health_problem']), 1, inplace = True)

In [ ]:
jigger_follow_ups_clean = jigger_details_b4aft[jigger_details_b4aft['year'] >= 2012]
jigger_details_wip.shape
#%store jigger_follow_ups_clean
jigger_follow_ups_clean.to_csv('codeclan/jigger_follow_ups_clean.csv', index = False, header=True)
#%store jigger_details_clean
jigger_details_clean = jigger_details_clean[jigger_details_clean['year'] >= 2012]
jigger_details_clean.to_csv('codeclan/jigger_details_clean.csv', index = False, header=True)

In [ ]:
jigger_follow_ups_clean.shape

In [ ]:
jigger_follow_ups_clean.columns

In [ ]:
jigger_follow_ups_clean.year.head()

In [ ]:
jigger_details_wip.employmentStatus.unique()

In [ ]:
jigger_follow_ups_clean.simd_d.unique()